## C953 Video Game Recommendation System

### Packages

Packages used in the program are imported. The display of the notebook is also adjusted to allow for a larger output window.

In [2]:
import pandas as pd
import ipywidgets as widgets
from sklearn.neighbors import NearestNeighbors
from IPython.display import display, clear_output, Markdown, HTML
import matplotlib.pyplot as plt
import warnings
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from wordcloud import WordCloud

display(HTML('<style>.container {width: 70% !important}</style>'))
display(HTML("<style>div.output_scroll { height: 77em; }</style>"))


### Data Loading

The datasets used are steam.csv, which is a dataset of Steam games with genre and content tags and other data such as price, release date, etc. The steamspy_tag_data.csv is a dataset from SteamSpy with crowdsourced genre/content tags.

In [3]:
steam_df = pd.read_csv('steam.csv')
steam_df = steam_df[steam_df['english']==1]
steam_df.drop(['achievements', 'price', 'english'], axis=1)
user_tag_df = pd.read_csv('steamspy_tag_data.csv', index_col='appid')
display(steam_df.head(10).style.hide(axis="index").set_properties(**{'width': '100%', 'max-width': '800px', 'overflow-x': 'auto'}))
display(user_tag_df.head(10).style.hide(axis="index").set_properties(**{'width': '100%', 'max-width': '800px', 'overflow-x': 'auto'}))



appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.190000
20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.990000
30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.990000
40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.990000
50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat enabled,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.990000
60,Ricochet,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,2758,684,175,10,5000000-10000000,3.990000
70,Half-Life,1998-11-08,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Online Multi-Player;Steam Cloud;Valve Anti-Cheat enabled,Action,FPS;Classic;Action,0,27755,1100,1300,83,5000000-10000000,7.190000
80,Counter-Strike: Condition Zero,2004-03-01,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,12120,1439,427,43,10000000-20000000,7.190000
130,Half-Life: Blue Shift,2001-06-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player,Action,FPS;Action;Sci-fi,0,3822,420,361,205,5000000-10000000,3.990000
220,Half-Life 2,2004-11-16,1,Valve,Valve,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading Cards;Captions available;Partial Controller Support;Steam Cloud;Includes Source SDK,Action,FPS;Action;Sci-fi,33,67902,2419,691,402,10000000-20000000,7.190000


### Preprocessing of Data

The data is processed, changing all strings to lower case and changing certain characters so that both steamspy tagging columns in each dataset will match. The steam.csv dataset is converted into a matrix containing only the appid used as a key, and columns representing each genre assigning values of 1 or 0 depending on whether the game has the tag or not. The matrix_df is then multiplied with the user_tag_df containing crowdsourced tagging data, removing tagging data if the game wasn't tagged as such in the original steam.csv dataset 

In [4]:
matrix_df = steam_df['steamspy_tags'].str.lower().str.replace(" ", "_").str.replace("'", "").str.replace("-", "_").str.get_dummies(';')
tag_df = steam_df[['appid', 'steamspy_tags']].assign(steamspy_tags=steam_df['steamspy_tags'].str.lower().str.replace(" ", "_").str.replace("'", "").str.replace("-", "_"))
tag_df = tag_df.assign(tag=tag_df['steamspy_tags'].str.split(';')).explode('tag')
matrix_df = tag_df.pivot_table(index='appid', columns='tag', aggfunc=len, fill_value=0)
matrix_df.columns = matrix_df.columns.droplevel()
user_tag_df = user_tag_df[user_tag_df.index.isin(matrix_df.index)]
merged_df = matrix_df.multiply(user_tag_df)
merged_df=merged_df.fillna(0)
merged_df = merged_df[merged_df.index.isin(steam_df['appid'])]
display(merged_df.head(10).style.set_properties(**{'width': '100%', 'max-width': '800px', 'overflow-x': 'auto'}))



### K-Nearest Neighbors model is created and fitted using cosine similarity scoring

This uses the merged_df dataframe created in the data preprocessing section above

In [5]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(merged_df)


NearestNeighbors(algorithm='brute', metric='cosine')

### Recommendation Algorithm

The algorithm generates top 10 recommendations of a given game by running the K-NN model n=100, and returning the top 10 results as a data frame with the game ID, name, and the cosine similarity score.

In [6]:
def recommend_game(game_id):
    # Find k nearest neighbors of the input game
    distances, indices = model_knn.kneighbors(merged_df.loc[[game_id]], n_neighbors=100)

    # Get the indices of k nearest neighbors
    similar_games = merged_df.iloc[indices[0][1:], :].index

    # Get cosine similarity scores for the recommended games
    similarity_scores = 1 - distances[0][1:]

    # Get data for recommended games
    similar_games_df = steam_df.loc[steam_df['appid'].isin(similar_games), ['appid', 'name']]

    # Sort the recommended games by cosine similarity
    similar_games_df['similarity'] = similarity_scores
    similar_games_df = similar_games_df.sort_values(by='similarity', ascending=False).reset_index(drop=True)


    return similar_games_df.head(10)

### Graphing Functions

Three functions create the following visualizations given the results of the game recommendation function:

- A horizontal bar plot showing the similarity scores of recommended games.
- A network graph showing the relationships between the recommended games and tags
- A word cloud showing the most frequent tags that appear in the 

In [7]:
def generate_similarity_barplot(recommended_games):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.barh(recommended_games['name'], recommended_games['similarity'], color='steelblue')
    ax.set_title('Cosine Similarity Scores of Recommended Games')
    ax.set_xlabel('Cosine Similarity Score')
    ax.set_ylabel('Game Title')
    ax.autoscale()
    plt.show()
def generate_network_graph(recommended_games):
    G = nx.Graph()
    game_color = 'blue'
    tag_color = 'red'
    
    for game_id in recommended_games['appid']:
        game_tags = tag_df[tag_df['appid'] == game_id]['tag'].values.tolist()
        game_name = steam_df[steam_df['appid'] == game_id]['name'].values[0]
        G.add_node(game_name, node_color=game_color)
        for tag in game_tags:
            G.add_node(tag, node_color=tag_color)
            G.add_edge(game_name, tag)

    pos = nx.spring_layout(G)
    fig, ax = plt.subplots(figsize=(10, 10))
    nx.draw_networkx_nodes(G, pos, nodelist=[n for n in G.nodes() if G.nodes[n]['node_color'] == game_color], node_color=game_color, node_size=500, alpha=0.8, ax=ax)
    nx.draw_networkx_nodes(G, pos, nodelist=[n for n in G.nodes() if G.nodes[n]['node_color'] == tag_color], node_color=tag_color, node_size=100, alpha=0.8, ax=ax)
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5, ax=ax)
    nx.draw_networkx_labels(G, pos, font_size=10, font_family='sans-serif', ax=ax)

    ax.set_title('Network Graph of Recommended Games and Tags')

    plt.show()

def generate_word_cloud(recommended_games):
    tags = []
    for game_id in recommended_games['appid']:
        game_tags = tag_df[tag_df['appid'] == game_id]['tag'].values.tolist()
        tags.extend(game_tags)

    wordcloud = WordCloud(background_color="white").generate(' '.join(tags))

    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()


### The Application

The algorithm can be interacted with via a few user interface objects below. You can either input the game name (copied from the steam.csv) into the search bar and click 'Run' to run the algorithm and show the visualizations, or click 'I'm Feeling Lucky', which will find a random game from the dataset to run the algorithm against.

In [8]:
input_widget = widgets.Text(description='Game')
button1 = widgets.Button(description='Run')
button2 = widgets.Button(description="I'm Feeling Lucky")
vbox = widgets.VBox()
output_widget = widgets.Output()
graph_output_widget = widgets.Output()

#Ignores warnings from missing fonts/glyphs
warnings.filterwarnings("ignore", category=UserWarning)

def on_button1_click(b):
    with output_widget:
        try:
            output_widget.clear_output(wait = True)
            input_value = input_widget.value
            game_input = steam_df.loc[steam_df['name'] == input_value].iloc[0]
            run_algorithm(game_input)
        except IndexError as e:
            error_string = Markdown(f'__An error occurred. The game must be inside of the Steam dataset used in the notebook (steam.csv)__ <br> <br> Error:{e}')
            output1 = error_string
            display(output1)
def on_button2_click(b) :
    with output_widget:
        output_widget.clear_output(wait=True)
        random_row = steam_df.sample(n=1).iloc[0]
        random_game_name = random_row.loc['name']
        run_algorithm(random_row)
        
def run_algorithm(game_input):
            game_name = game_input['name']
            output1 = Markdown(f'Recommendations for {game_name}:')
            output2 = recommend_game(game_input['appid'])
            display(output1, output2)
            generate_similarity_barplot(output2)
            generate_network_graph(output2)
            generate_word_cloud(output2)


        
        
button1.on_click(on_button1_click)
button2.on_click(on_button2_click)
display(vbox, input_widget, button1, button2, output_widget, graph_output_widget)

VBox()

Text(value='', description='Game')

Button(description='Run', style=ButtonStyle())

Button(description="I'm Feeling Lucky", style=ButtonStyle())

Output()

Output()